In [1]:
import io
import xml.etree.ElementTree as ET
import pandas as pd
import helper
from sklearn.feature_extraction.text import CountVectorizer
import gensim
import numpy as np
import os
from gensim import corpora, models, similarities 
from gensim.corpora.mmcorpus import MmCorpus

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
%load_ext autotime

Installed autotime.py. To use it, type:
  %load_ext autotime


/home/jshoun01/.virtualenvs/py27/local/lib/python2.7/site-packages/IPython/core/magics/extension.py:47: UserWarning: %install_ext` is deprecated, please distribute your extension(s)as a python packages.
  "as a python packages.", UserWarning)


In [4]:
#%unload_ext autotime

time: 305 ms


In [5]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

time: 651 ms


In [6]:
ls data

data@
time: 604 ms


In [7]:
data_dir='data'

time: 306 ms


## Normalizer

In [8]:
ag_preprocesser = helper.Normalizer(stem=False\
                                 ,stopword_removal=True\
                                 ,strip_html_tags=True
                                   ,filter_parts_of_speech=True
                                    ,min_word_length=1
                                   )

time: 428 ms


## Loading Streaming

In [9]:
post_file=os.path.join(data_dir,'Posts.xml')
posts_corpus = helper.XmlCorpus(post_file
                                #,clip_docs=100000
                                ,clip_docs=1000
                                ,body_field='Body'
                                ,normalizer=ag_preprocesser)
posts_corpus.finalize_dictionary()

time: 37.2 s


In [10]:
sample_msg='<p>A Vista virtual machine I use only has a 10 GB virtual hard disk -- and I\'m worried about it running out of space over time.</p>\n\n<p>I used <a href="http://www.jam-software.com/treesize_free/">TreeSize</a> to check for the culprits..</p>\n\n<p><img src="http://i.stack.imgur.com/uGfhQ.png" alt="enter image description here"></p>\n\n<p>And the primary culprit is that <strong><code>/winsxs</code> or "Windows Side-By-Side" folder</strong>.</p>\n\n<p><img src="http://i.stack.imgur.com/DWYOo.png" alt="enter image description here"></p>\n\n<p>What is the Windows Side By Side folder? Well, that\'s <a href="http://blogs.technet.com/askcore/archive/2008/09/17/what-is-the-winsxs-directory-in-windows-2008-and-windows-vista-and-why-is-it-so-large.aspx">explained pretty well here</a>.</p>\n\n<blockquote>\n  <p>All of the components in the operating system are found in the WinSxS folder \u2013 in fact we call this location the component store.  Each component has a unique name that includes the version, language, and processor architecture that it was built for.  The WinSxS folder is the only location that the component is found on the system, all other instances of the files that you see on the system are \u201cprojected\u201d by hard linking from the component store.  Let me repeat that last point \u2013 there is only one instance (or full data copy) of each version of each file in the OS, and that instance is located in the WinSxS folder.   So looked at from that perspective, the WinSxS folder is really the entirety of the whole OS, referred to as a "flat" in down-level operating systems.  This also accounts for why you will no longer be prompted for media when running operations such as System File Checker (SFC), or when installing additional features and roles.</p>\n  \n  <p>That explains why the folder starts off big, but not why it gets larger over time \u2013 the answer to that question is servicing.   In previous versions of Windows the atomic unit of servicing was the file, in Windows Vista it\u2019s the component.  When we update a particular binary we release a new version of the whole component, and that new version is stored alongside the original one in the component store.  The higher version of the component is projected onto the system, but the older version in the store isn\u2019t touched.  The reason for that is the third part of why the component store gets so large.</p>\n</blockquote>\n\n<p>Lots of other people <a href="http://blog.tiensivu.com/aaron/archives/1306-Demystifying-the-WinSxS-directory-in-Windows-XP,-Vista-and-Server-20032008.html">complaining about the size of the /winsxs folder, too</a>:</p>\n\n<blockquote>\n  <p>A coworker recently received his every two year work laptop refresh and was in the middle of loading his new laptop with Vista. For some reason, his hard drive C: was partitioned to only 25GB and the rest of the space reserved for a much larger drive D:.</p>\n  \n  <p>Anyway, after loading many Windows Updates, he was left with hardly any drive space left on C:, and for some reason, the WinSxS directory had ballooned to over 8GB.</p>\n  \n  <p>While I knew what the WinSxS (Windows Side-by-Side) directory was for, I wasn\'t quite sure why that directory would explode in size so fast with normal operating system updates.</p>\n</blockquote>\n\n<p>I don\'t really care about having all these duplicate files for safety, but I <em>do</em> care about the VM exploding due to an out of disk space error. So.. <strong>can /winsxs be safely pared down in any way?</strong> <a href="http://blogs.msdn.com/jonwis/archive/2007/01/02/deleting-from-the-winsxs-directory.aspx">It doesn\'t seem that it can be</a>.. </p>\n\n<blockquote>\n  <p><strong>Administrators should not, for any reason, take it upon themselves to clean out the /winsxs directory</strong> - doing so may prevent Windows Update and MSI from functioning properly afterwards.  Preventing accidental deletion from the directory is accomplished by putting a strong security descriptor on the directory that inherits to its children.</p>\n</blockquote>\n\n<p>If this /winsxs folder gets any larger, I\'m going to have to re-image this virtual machine.. which I <em>really</em> don\'t want to do!</p>\n'

ag_preprocesser.process(sample_msg)[0:5]

[u'Vista', u'virtual', u'machine', u'use', u'GB']

time: 23.3 ms


In [11]:
"""
num_documents_processed=0
counter=0
for vector in posts_corpus: # load one vector into memory at a time
    print ("Item %s"%(counter))
    print(vector)
    print()
    counter=counter+1
"""

'\nnum_documents_processed=0\ncounter=0\nfor vector in posts_corpus: # load one vector into memory at a time\n    print ("Item %s"%(counter))\n    print(vector)\n    print()\n    counter=counter+1\n'

time: 736 ms


In [12]:
print ("Number of docs in collection: %s"%(len(posts_corpus)))

Number of docs in collection: 1001
time: 729 ms


In [13]:
#gensim.models.LdaMulticore
"""
lda = gensim.models.LdaModel(corpus, num_topics=5, 
                            id2word=dictionary, 
                            update_every=5, 
                            chunksize=10000,
                            passes=100)
"""

num_topics = 20

#lda = gensim.models.LdaModel(posts_corpus, num_topics=5, 
#                            id2word=posts_corpus.dictionary
#                                ,passes=2
#                                )

lda = gensim.models.LdaMulticore(posts_corpus, num_topics=num_topics, 
                            id2word=posts_corpus.dictionary
                                 ,passes=3
                                )

1000
1000
1000
time: 2min 4s


In [14]:
lda.show_topics(num_topics)

[u'0.019*fan + 0.018*CPU + 0.012*use + 0.011*speed + 0.011*like + 0.010*lot + 0.010*tab + 0.009*drive + 0.008*much + 0.008*time',
 u'0.015*Windows + 0.013*file + 0.011*also + 0.011*work + 0.010*get + 0.009*system + 0.009*try + 0.009*network + 0.008*different + 0.008*USB',
 u'0.023*search + 0.019*use + 0.012*used + 0.010*network + 0.010*time + 0.009*system + 0.008*laptop + 0.008*Windows + 0.007*would + 0.007*server',
 u'0.052*Windows + 0.017*install + 0.014*version + 0.013*Vista + 0.013*like + 0.012*http + 0.010*folder + 0.010*RC + 0.010*clean + 0.010*disk',
 u'0.025*use + 0.021*screen + 0.014*also + 0.012*memory + 0.012*would + 0.011*like + 0.009*card + 0.008*email + 0.008*using + 0.007*one',
 u'0.021*Windows + 0.020*use + 0.018*Vista + 0.012*user + 0.012*need + 0.010*run + 0.010*like + 0.010*PC + 0.007*tried + 0.007*home',
 u'0.016*use + 0.010*Mac + 0.010*like + 0.010*software + 0.009*works + 0.009*computer + 0.009*monitor + 0.009*sync + 0.009*get + 0.008*one',
 u'0.016*drive + 0.014*

time: 4.24 ms


In [15]:
topics_matrix = lda.show_topics(formatted=False, num_words=20)
topics_matrix = np.array(topics_matrix)

topic_words = topics_matrix[:,:,1]
for i in topic_words:
    print([str(word) for word in i])
    print()

['USB', 'drive', 'use', 'know', 'work', 'using', 'usually', 'Ubuntu', 'boot', 'like', 'available', 'file', 'things', 'folder', 'DVD', 'problem', 'hard', 'partition', 'think', 'Windows']
()
['Windows', 'install', 'version', 'Vista', 'like', 'http', 'folder', 'RC', 'clean', 'disk', 'use', 'upgrade', 'backup', 'one', 'OS', 'first', 'need', 'would', 'want', 'see']
()
['Windows', 'using', 'Gmail', 'better', 'like', 'power', 'use', 'application', 'set', 'settings', 'need', 'manager', 'media', 'lose', 'see', 'cores', 'windows', 'computer', 'would', 'XP']
()
['use', 'Mac', 'like', 'software', 'works', 'computer', 'monitor', 'sync', 'get', 'one', 'Google', 'good', 'device', 'much', 'problem', 'video', 'time', 'working', 'new', 'would']
()
['use', 'screen', 'also', 'memory', 'would', 'like', 'card', 'email', 'using', 'one', 'server', 'used', 'Media', 'Center', 'version', 'different', 'http', 'IMAP', 'could', 'monitor']
()
['would', 'Flash', 'use', 'browser', 'network', 'using', 'file', 'third-pa

In [16]:
lda.show_topics()

[u'0.025*use + 0.021*screen + 0.014*also + 0.012*memory + 0.012*would + 0.011*like + 0.009*card + 0.008*email + 0.008*using + 0.007*one',
 u'0.052*Windows + 0.017*install + 0.014*version + 0.013*Vista + 0.013*like + 0.012*http + 0.010*folder + 0.010*RC + 0.010*clean + 0.010*disk',
 u'0.017*use + 0.012*drive + 0.012*Ubuntu + 0.011*like + 0.011*Windows + 0.010*computer + 0.009*want + 0.009*Linux + 0.009*OS + 0.009*one',
 u'0.015*Windows + 0.013*file + 0.011*also + 0.011*work + 0.010*get + 0.009*system + 0.009*try + 0.009*network + 0.008*different + 0.008*USB',
 u'0.016*drive + 0.014*use + 0.013*computer + 0.013*files + 0.012*like + 0.010*work + 0.010*system + 0.009*video + 0.009*one + 0.008*want',
 u'0.019*fan + 0.018*CPU + 0.012*use + 0.011*speed + 0.011*like + 0.010*lot + 0.010*tab + 0.009*drive + 0.008*much + 0.008*time',
 u'0.016*use + 0.010*Mac + 0.010*like + 0.010*software + 0.009*works + 0.009*computer + 0.009*monitor + 0.009*sync + 0.009*get + 0.008*one',
 u'0.017*would + 0.017*F

time: 678 ms


In [17]:
model_serialized_path='lda_topic_%s.lda'%(num_topics)
lda.save(model_serialized_path) 

time: 601 ms


In [18]:
lda = gensim.models.LdaMulticore.load(model_serialized_path)

time: 696 ms


In [19]:
lda.show_topics()

[u'0.017*use + 0.012*drive + 0.012*Ubuntu + 0.011*like + 0.011*Windows + 0.010*computer + 0.009*want + 0.009*Linux + 0.009*OS + 0.009*one',
 u'0.021*Windows + 0.020*use + 0.018*Vista + 0.012*user + 0.012*need + 0.010*run + 0.010*like + 0.010*PC + 0.007*tried + 0.007*home',
 u'0.015*would + 0.013*Windows + 0.013*new + 0.013*PC + 0.011*one + 0.010*could + 0.010*use + 0.009*drive + 0.008*data + 0.008*hard',
 u'0.023*search + 0.019*use + 0.012*used + 0.010*network + 0.010*time + 0.009*system + 0.008*laptop + 0.008*Windows + 0.007*would + 0.007*server',
 u'0.025*use + 0.021*screen + 0.014*also + 0.012*memory + 0.012*would + 0.011*like + 0.009*card + 0.008*email + 0.008*using + 0.007*one',
 u'0.013*use + 0.011*option + 0.009*also + 0.009*Windows + 0.009*driver + 0.009*menu + 0.009*one + 0.009*install + 0.008*kernel + 0.008*could',
 u'0.024*use + 0.016*computer + 0.016*using + 0.015*work + 0.012*way + 0.012*want + 0.012*get + 0.010*one + 0.010*email + 0.009*web',
 u'0.023*use + 0.020*Windows 

time: 2.98 ms
